<a href="https://colab.research.google.com/github/TheChanRProject/Senior-Thesis/blob/master/nb/RagaCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://github.com/TheChanRProject/ragaImages.git

Cloning into 'ragaImages'...
remote: Enumerating objects: 796, done.
remote: Counting objects: 100% (796/796), done.
remote: Compressing objects: 100% (794/794), done.
remote: Total 796 (delta 0), reused 793 (delta 0), pack-reused 0
Receiving objects: 100% (796/796), 16.10 MiB | 33.51 MiB/s, done.


In [2]:
ls

ragaImages/  sample_data/


In [3]:
cd ragaImages

/content/ragaImages


In [4]:
ls

numbers.py  README.md  testing/  training/


In [6]:
!python numbers.py

124
233
180
165
13
25
22
18


In [2]:
!git clone https://github.com/TheChanRProject/ragaImages.git 
 

Cloning into 'ragaImages'...
remote: Enumerating objects: 796, done.
remote: Counting objects: 100% (796/796), done.
remote: Compressing objects: 100% (794/794), done.
remote: Total 796 (delta 0), reused 793 (delta 0), pack-reused 0
Receiving objects: 100% (796/796), 16.10 MiB | 15.82 MiB/s, done.


In [3]:
cd ragaImages && !python numbers.py

[Errno 2] No such file or directory: 'ragaImages && !python numbers.py'
/content


In [4]:
cd ragaImages

/content/ragaImages


In [5]:
!python numbers.py

233
180
124
165
25
22
13
18


In [17]:
!git pull

remote: Enumerating objects: 13, done.
remote: Counting objects: 100% (13/13), done.
remote: Compressing objects: 100% (9/9), done.
remote: Total 12 (delta 2), reused 11 (delta 1), pack-reused 0
Unpacking objects: 100% (12/12), done.
From https://github.com/TheChanRProject/ragaImages
   fc0035b..709e3b0  master     -> origin/master
Updating fc0035b..709e3b0
Fast-forward
 csv/miml_labels_2.csv                             | 781 ++++++++++++++++++++++
 testing/bhairav/bhairav1_0.png => data/1.png      | Bin
 testing/bhairav/bhairav2_2.png => data/10.png     | Bin
 training/bhairav/bhairav6_8.png => data/100.png   | Bin
 training/bhairav/bhairav4_26.png => data/101.png  | Bin
 training/bhairav/bhairav7_4.png => data/102.png   | Bin
 training/bhairav/bhairav4_27.png => data/103.png  | Bin
 training/bhairav/bhairav6_9.png => data/104.png   | Bin
 training/bhairav/bhairav4_28.png => data/105.png  | Bin
 training/bhairav/bhairav8_2.png => data/106.png   | Bin
 training/bhairav/bhairav4_29.png 

In [0]:
import tensorflow as tf 

In [18]:
ls

csv/  data/  file_rename.py  label_csv.py  README.md


In [0]:
# Importing Tensorflow's keras wrapper 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization
from tensorflow.keras import regularizers, optimizers 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np
import pandas as pd 

In [25]:
df = pd.read_csv("csv/miml_labels_2.csv") 
df.drop(list(df.columns)[0], axis=1, inplace=True)
df.head()
df["raga"]=df["raga"].apply(lambda x:x.split(","))
df.head()

,file_name,raga
0,1.png,[bhairav]
1,2.png,[bhairav]
2,3.png,[bhairav]
3,4.png,[bhairav]
4,5.png,[bhairav]


In [0]:
datagen=ImageDataGenerator(rescale=1./255.)
test_datagen=ImageDataGenerator(rescale=1./255.)

In [27]:
train_generator=datagen.flow_from_dataframe(
dataframe=df[:680],
directory="./data",
x_col="file_name",
y_col="raga",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["bhairav", "bhup", "des", "yaman"],
target_size=(432,288))

valid_generator=test_datagen.flow_from_dataframe(
dataframe=df[680:730],
directory="./data",
x_col="file_name",
y_col="raga",
batch_size=32,
seed=42,
shuffle=True,
class_mode="categorical",
classes=["bhairav", "bhup", "des", "yaman"],
target_size=(432,288)) 

test_generator=test_datagen.flow_from_dataframe(
dataframe=df[730:],
directory="./data",
x_col="file_name",
batch_size=1,
seed=42,
shuffle=False,
class_mode=None,
target_size=(432,288)) 

Found 680 images belonging to 4 classes.
Found 50 images belonging to 4 classes.
Found 50 images.


In [0]:
classifier = Sequential() 
classifier.add(Conv2D(32, (3, 3), padding='same', input_shape = (432, 288, 3), activation = 'relu')) 
classifier.add(Conv2D(32, (3, 3), activation = 'relu')) 
classifier.add(MaxPooling2D(pool_size = (2, 2)))
classifier.add(Dropout(rate=1-0.25))
classifier.add(Conv2D(64,(3,3), padding="same", activation="relu"))
classifier.add(Conv2D(64, (3,3), activation="relu")) 
classifier.add(MaxPooling2D(pool_size=(2,2))) 
classifier.add(Dropout(rate=1-0.25)) 
classifier.add(Flatten())
classifier.add(Dense(units = 512, activation = 'relu'))
classifier.add(Dropout(rate=1-0.5)) 
classifier.add(Dense(units = 4, activation = 'softmax')) 
classifier.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])


In [44]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size
STEP_SIZE_TEST=test_generator.n//test_generator.batch_size
classifier.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=10
)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
22/22 [==============================] - 18s 835ms/step - loss: 3.7171 - acc: 0.2941 - val_loss: 1.4156 - val_acc: 0.0000e+00
Epoch 2/10
22/22 [==============================] - 10s 433ms/step - loss: 1.3825 - acc: 0.2882 - val_loss: 1.3968 - val_acc: 0.0000e+00
Epoch 3/10
22/22 [==============================] - 10s 433ms/step - loss: 1.3766 - acc: 0.2985 - val_loss: 1.4198 - val_acc: 0.0000e+00
Epoch 4/10
22/22 [==============================] - 10s 440ms/step - loss: 1.3726 - acc: 0.3088 - val_loss: 1.4170 - val_acc: 0.0000e+00
Epoch 5/10
22/22 [==============================] - 10s 443ms/step - loss: 1.3551 - acc: 0.3426 - val_loss: 1.3585 - val_acc: 0.0400
Epoch 6/10
22/22 [==============================] - 10s 446ms/step - loss: 1.3025 - acc: 0.3985 - val_loss: 1.4321 - val_acc: 0.1000
Epoch 7/10
22/22 [==============================] - 10s 449ms/step - loss: 1.1649 - acc: 0.4809 - val_loss: 1.3719 - val_acc: 0.3000
Epoch

In [48]:
test_generator.reset()
pred=classifier.predict_generator(test_generator,
steps=STEP_SIZE_TEST,
verbose=1) 

pred_bool = (pred >0.5)

predictions=[]
labels = train_generator.class_indices
labels = dict((v,k) for k,v in labels.items())
for row in pred_bool:
    l=[]
    for index,cls in enumerate(row):
        if cls:
            l.append(labels[index])
    predictions.append(",".join(l))
filenames=test_generator.filenames
results=pd.DataFrame({"file_name":filenames,
                      "predictions":predictions})



50/50 [==============================] - 1s 11ms/step


In [50]:
results

,file_name,predictions
0,731.png,
1,732.png,yaman
2,733.png,
3,734.png,
4,735.png,
5,736.png,
6,737.png,
7,738.png,
8,739.png,
9,740.png,yaman
